**Übung Mustererkennung** *WS 2022/23* -- *K. Brandenbusch,  Gernot A. Fink* -- *Technische Universität Dortmund, Lehrstuhl XII, Mustererkennung in eingebetteten Systemen*
___
# Aufgabe 8 (optional): Support Vector Machines - MNIST

Die Leistungsfähigkeit der Support Vector Machines soll nun für das realistischere Szenario der Zeichenerkennung untersucht werden. 
Hierzu soll ein komplettes Klassifikationssystem für den MNIST-Datensatz erstellt werden.

---

Zuerst müssen die benötigten Module importiert werden.

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys
import numpy as np
from sklearn import svm

# Uebergeordneten Ordner zum Pfad hinzufuegen, damit das common Package importiert werden kann
if '..' not in sys.path:
    sys.path.append('..')

from common.classification import ClassificationEvaluator
from common.data_provider import DataProvider

---
## Lineare SVM auf Originaldaten

Laden Sie zunächst die MNIST-Daten in Originalrepräsentation (784-D) und trainieren Sie eine lineare SVM mit den Standardparametern von sklearn.

---
## Lineare SVM auf PCA-reduzierten Daten

Verwenden Sie an Stelle der 784-dimensionalen Originaldaten die Merkmalsrepraesentationen aus Tag 3. Die jeweiligen SVMs sollen auf der Basis einer linearen SVM mit soft-margin erstellt werden. Vergleichen Sie die Ergebnisse.

---
## Kernel SVM auf PCA-reduzierten Daten

Variieren Sie die verwendete Kernelfunktion und vergleichen Sie Ihre Ergebnisse.

---
## Parametersuche

Evaluieren Sie, wie sich Veränderungen der jeweiligen Kernelparameter auf die entsprechenden Klassifikationsergebnisse auswirken.